In [17]:
%matplotlib inline
import matplotlib
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
#import seaborn
from matplotlib import colors
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import random
from scipy.sparse import diags
from numba import jit
from numba import cuda

Diff=3e15
lambdacn=0.5
nteps=1000
deltat=5.0e8
deltat2=10000.*365*3600*24
Mdotdisk=1e-5
Rini=1.
rhoc=3.2
rhos=3.2 
rhos2=3.2 
psize=1e-4
adiagrad=0.28
Zgr=1e-8
T0=150.
d0=1e-11
ggrav=6.674e-8
stephboltz=5.67e-5
boltz=1.38e-16
random.seed(42)
vs= 2.992e-23
mudisk=2.34 * 1.67e-24
mcore=1.1*5.97e27
xfinind=0
xfin=[]
mu = np.empty(nteps)
mu.fill(2.34 * 1.67e-24)
timeabs=[]

dMassice=np.zeros(nteps)
radgradtot2=np.zeros(nteps)
mugrad=np.empty(nteps)
ledouxgrad=np.empty(nteps)

##@cuda.jit
def g(y, x):
    global radgradtot
    global adiagrad
    global posx

#    posx=nteps-int(round(0.5*x/(501187233.627)))-1    
    xx = np.logspace(12,8.7,nteps)
    posx = find_nearest(xx,x)    
    radgradtot=(radgradp*(y[0]/((y[1])**4.))*((1e-8*(((y[0]*mu[posx])/(y[1]*boltz))**(2./3))*(y[1]**(3.))) +
         ((min(2.,Qep*y[1]))*3*Zgr/(4*rhos*psize))))
    adiagrad=0.28
    radgradtotind.append(radgradtot)
    adiagradind.append(adiagrad)
    
    y0 = -(ggrav*mcore*mu[posx]/boltz)*(y[0]/y[1])*(1/(x**2))
    y1 = -y0*(y[1]/y[0])*(-1)*min(adiagrad,radgradtot)         
    return y0, y1

#@cuda.jit
def cs(Temp,pos):
    cs=np.sqrt(1.6*boltz*Temp/mu[pos])
    return cs

#@cuda.jit
def vdrag(R,pos):
    vdrag=rhos2*R*((Pressure[pos+1]-Pressure[pos-1])/(x[pos+1]-x[pos-1]))/(cs(Temp,pos)*(Density[pos]*d0)**2)
    return vdrag

#@cuda.jit
def pp(Temp):
#        pp=1.013e6*np.exp(-5940/Temp+15.6)   
    pp=1.013e6*10**(-24605/Temp+13.176)
    return pp 
    
#@cuda.jit
def drdt(R,Temp):   
    A=(vs*0.63/(np.sqrt(2*3.14*boltz*(4./3)*3.14*rhos)))*((pp(Temp))/(np.sqrt(Temp)))
    k1=deltat*A*(1/(np.sqrt(R**3)))
#    k2=deltat*A*(1/(np.sqrt((R+k1/2)**3)))
#    k3=deltat*A*(1/(np.sqrt((R+k2/2)**3)))
#    k4=deltat*A*(1/(np.sqrt((R+k3/2)**3)))
#    drdt=k1/6+k2/3+k3/3+k4/6
    drdt=k1
    return drdt

##@cuda.jit 
def subR(Rini):   
    global Temp
    R=Rini
    timeind=1
    num=1
    while (num <= (nteps-3)):
        while (timeind <= 99999999):
            if (R > 0.):
                Temp=sol[num,1]
                R=R-drdt(R, Temp)                
                time=timeind*deltat
                travdist=abs(vdrag(R,num)*time)        
                if (travdist >= abs(x[num]-x[(num+1)])):
                    if (num <= (nteps-3)):
                        num=num+1
                    timeind=1.
            if (R <= 0.):
                break
        if (R <= 0.):
            break            
            timeind=timeind+1
    return num

#@cuda.jit
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

#@cuda.jit
def CNX(X):
    b=Diff*deltat/(xgrad**2)
    b1=lambdacn*b
    b2=(1-lambdacn)*b
    a=2*Diff*deltat/(2*xgrad*x)
    a1=lambdacn*a
    a2=(1-lambdacn)*a
    A=np.zeros(sizemesh)
    B=np.zeros(sizemesh)
    C=np.zeros(sizemesh)
    D=np.ones(sizemesh)
    A[:]=-a1-b1
    B[:]=1+2*b1
    C[:]=a1-b1

    Ap=A[1:sizemesh]
    Ap=np.append(Ap,0 )
    A=Ap
    C=np.insert(C,0,0 )
    C=np.delete(C, sizemesh)

    B[0]=1.
    C[0]=-1.
    C[1]=-1.
    D[0]=0.

    A[sizemesh-1]=1.
    A[sizemesh-2]=1.
    B[sizemesh-1]=-1.
    D[sizemesh-1]=0.


    M = sp.sparse.dia_matrix(([B, C, A], [0, 1, -1]), shape=(sizemesh, sizemesh))
    M = M.todense()

    for i in range (1,sizemesh-1,1):
        D[i]=((a2[i]+b2[i])*X[i+1]+(1-2*b2[i])*X[i]+(b2[i]-a2[i])*X[i-1])

    X=sp.sparse.linalg.spsolve(M,D)
    
    return X



for j in range(1,99999,1):
    timeabs.append(j*deltat2)
    
    mcore=(mcore+Mdotdisk*(5.97e27/(365*3600*24))*deltat2)

    init = (d0*boltz*T0)/(mudisk) , T0
    Rcore=(3*mcore/(4*3.14*rhoc))**(1./3)
    Bondi = (ggrav*mcore/(boltz*T0/mudisk))/Rcore
    mdottot=Mdotdisk*(5.97e27/(365*3600*24))
    radgradp=(3*mdottot)/(64*3.14*stephboltz*Rcore)
    Qep=(0.3*2*3.14*psize)/(0.289)
    dMassg=np.zeros(nteps)
    x = np.logspace(12,8.7,nteps)
    sizemesh=len(x)
    xgrad=np.gradient(x)
    
    radgradtotind=[]
    adiagradind=[]


    sol=odeint(g, init, x)  

    Density = (((sol[:,0]*mu)/(boltz*sol[:,1])))
    Temperature = (sol[:,1])/T0
    Pressure=sol[:,0]

#    for i in range(1,nteps,1):
#        dMassg[i]=((4/3)*3.14*Density[i]*((x[i-1]-x[i])**3))
    dMassg=((4/3)*3.14*Density*((np.diff(x,n=0))**3))        
        
        
    if (Rini > 0):
        xfin.append(x[subR(Rini)]/Rcore)
#        dMassice[subR(Rini)]=dMassice[subR(Rini)] + (4/3)*3.14*rhos*(Rini)**3
        dMassice[subR(Rini)]=dMassice[subR(Rini)] + 1e-5*(5.97e27)*deltat2
#        print subR(Rini),Rini
    mu=(1./(((dMassg/(dMassg+dMassice))/2.2)+(((dMassice)/(dMassg+dMassice))/18.))) * 1.67e-24
#    print mu
    mu[np.isnan(mu)] = 2.34 * 1.67e-24#
    mu=CNX(mu)
#    print mu

    for i in range(1,nteps-1,1):
        radgradtot2[i]=((radgradp*(sol[i,0]/((sol[i,1])**4.))*((1e-8*(((sol[i,0]*mu[i])/(sol[i,1]*boltz))**(2./3))*(sol[i,1]**(3.))) +
                         ((min(2.,Qep*sol[i,1]))*3*Zgr/(4*rhos*psize)))))
    #print mu2[np.where(mu2>2.2)]
    mugrad=(np.gradient(np.log(mu)))/(np.gradient(np.log(Pressure)))
#    mugrad=(-1.)*np.abs(mugrad)
    ledouxgrad=(radgradtot2-mugrad)
#    print len(radgradtot2), len(mugrad)

    
    #print (adiagrad-ledouxgrad)
#    print len(radgradtotind), len(adiagradind),len(radgradtot2)
   
#    print j, mcore/5.97e27
    if (mcore >= 5*(5.97e27)):
        break
#print CNX()
    

TypingError: Caused By:
Traceback (most recent call last):
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 228, in run
    stage()
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 442, in stage_nopython_frontend
    self.locals)
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/compiler.py", line 793, in type_inference_stage
    infer.build_constraint()
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 723, in build_constraint
    self.constrain_statement(inst)
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 882, in constrain_statement
    self.typeof_assign(inst)
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 940, in typeof_assign
    self.typeof_global(inst, inst.target, value)
  File "/home/mad/anaconda2/lib/python2.7/site-packages/numba/typeinfer.py", line 1034, in typeof_global
    typ = self.resolve_value

In [ ]:
#print mu
#print mugrad

adialim=0
adialimledoux=adialim
#print radgradtotind, adiagradind
#radgradtotind2=[]
#for i in range (len(radgradtotind)-1000,len(radgradtotind)-1,1):
#    radgradtotind2.append(radgradtotind[i])
#print     radgradtotind2

for i in range(1,len(radgradtot2)-1,1):
#    print radgradtotind2[i] , adiagradind[i]
    if (radgradtot2[i] < adiagrad):
        adialim=i
        
for i in range(adialim,len(radgradtot2),1):
#    print radgradtotind2[i] , adiagradind[i]
    if (ledouxgrad[i] > adiagrad):
        adialimledoux=i

#print     adialimledoux    
#for i in range(adialim,len(radgradtot2),1):
#    print   ledouxgrad[i]
#plt.plot(x/1e9, Density, color='b',label='Density')
plt.plot(x/Rcore, Temperature, color='r',label='Temp')
#plt.plot(x/Rcore, mu, color='r',label='mu')
plt.yscale('log')
plt.xscale('log')
plt.xticks(fontsize = 30, fontweight='bold')
plt.yticks(fontsize = 30, fontweight='bold')
plt.tick_params(which='minor', length=8)
plt.tick_params(which='major', length=16, color='r')
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.legend(loc=1, borderaxespad=0., fontsize=30)
ax = plt.gca()
legend = ax.get_legend()
#legend.legendHandles[0].set_color('blue')
legend.legendHandles[0].set_color('red')
plt.ylabel('X', fontsize=35, fontweight='bold')
plt.xlabel('R/R$_c$', fontsize=35, fontweight='bold')
#plt.axis((1,1000,1e-8,1))
plt.axis((1,1000,1,1e3))

#plt.axis((1,10,1e-24,1e-22))

plt.plot((Bondi, Bondi), (-1e-9,1e9), 'blue')
#plt.fill_betweenx(Pressure*1e-6,1,100, facecolor='blue')

plt.plot((x[adialim]/Rcore, x[adialim]/Rcore), (-1e-9,1e9), 'green')
plt.plot((x[adialimledoux]/Rcore, x[adialimledoux]/Rcore), (-1e-9,1e9), 'yellow')

plt.show()

#import seaborn
plt.scatter(timeabs,xfin, s=10*(4/3)*3.14*rhos*(Rini**3), color='r',label='Temp')
plt.yscale('log')
plt.xscale('log')
plt.xticks(fontsize = 30, fontweight='bold')
plt.tick_params(which='minor', length=8)
plt.tick_params(which='major', length=16, color='r')
#plt.axis((1,100,1e-6,1e-2))
plt.axis((1e11,1e14,1,10))
#plt.xticks(np.arange(1,200, 10.0))
plt.yticks(fontsize = 30, fontweight='bold')
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
#plt.legend(loc=2, borderaxespad=0., fontsize=30)
#ax = plt.gca()
#legend = ax.get_legend()
#legend.legendHandles[0].set_color('blue')
#legend.legendHandles[0].set_color('red')
plt.ylabel('R/Rc', fontsize=35, fontweight='bold')
plt.xlabel('Time', fontsize=35, fontweight='bold')
#plt.plot((x[len(radgradtotind2)-adialim]/Rcore, x[len(radgradtotind2)-adialim]/Rcore), (-1e-9,1e9), 'green')
plt.show()         
    

    

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import matplotlib.pyplot as plt
#import seaborn
from matplotlib import colors
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import random

nteps=1000
deltat=5.0e8
deltat2=10000.*365*3600*24
Mdotdisk=1e-5
Rini=1.
rhoc=3.2
rhos=3.2 
rhos2=3.2 
psize=1e-4
adiagrad=0.28
Zgr=1e-8
T0=150.
d0=1e-11
ggrav=6.674e-8
stephboltz=5.67e-5
boltz=1.38e-16
random.seed(42)
vs= 2.992e-23
mudisk=2.34 * 1.67e-24
mcore=1.1*5.97e27
xfinind=0
xfin=[]
mu = np.empty(nteps)
mu.fill(2.34 * 1.67e-24)
timeabs=[]

dMassice=np.zeros(nteps)
radgradtot2=np.zeros(nteps)
mugrad=np.empty(nteps)
ledouxgrad=np.empty(nteps)

def g(y, x):
    global radgradtot
    global adiagrad
    global posx

#    posx=nteps-int(round(0.5*x/(501187233.627)))-1    
    xx = np.logspace(12,8.7,nteps)
    posx = find_nearest(xx,x)    
    radgradtot=(radgradp*(y[0]/((y[1])**4.))*((1e-8*(((y[0]*mu[posx])/(y[1]*boltz))**(2./3))*(y[1]**(3.))) +
         ((min(2.,Qep*y[1]))*3*Zgr/(4*rhos*psize))))
    adiagrad=0.28
    radgradtotind.append(radgradtot)
    adiagradind.append(adiagrad)
    
    y0 = -(ggrav*mcore*mu[posx]/boltz)*(y[0]/y[1])*(1/(x**2))
    y1 = -y0*(y[1]/y[0])*(-1)*min(adiagrad,radgradtot)         
    return y0, y1

def cs(Temp,pos):
    cs=np.sqrt(1.6*boltz*Temp/mu[pos])
    return cs

def vdrag(R,pos):
    vdrag=rhos2*R*((Pressure[pos+1]-Pressure[pos-1])/(x[pos+1]-x[pos-1]))/(cs(Temp,pos)*(Density[pos]*d0)**2)
    return vdrag


def drdt(R,Temp):
    def pp(Temp):
#        pp=1.013e6*np.exp(-5940/Temp+15.6)   
        pp=1.013e6*10**(-24605/Temp+13.176)
        return pp    
    A=(vs*0.63/(np.sqrt(2*3.14*boltz*(4./3)*3.14*rhos)))*((pp(Temp))/(np.sqrt(Temp)))
    k1=deltat*A*(1/(np.sqrt(R**3)))
#    k2=deltat*A*(1/(np.sqrt((R+k1/2)**3)))
#    k3=deltat*A*(1/(np.sqrt((R+k2/2)**3)))
#    k4=deltat*A*(1/(np.sqrt((R+k3/2)**3)))
#    drdt=k1/6+k2/3+k3/3+k4/6
    drdt=k1
    return drdt

 
def subR(Rini):   
    global Temp
    R=Rini
    timeind=1
    num=1
    while (num <= (nteps-3)):
        while (timeind <= 99999999):
            if (R > 0.):
                Temp=sol[num,1]
                R=R-drdt(R, Temp)                
                time=timeind*deltat
                travdist=abs(vdrag(R,num)*time)        
                if (travdist >= abs(x[num]-x[(num+1)])):
                    if (num <= (nteps-3)):
                        num=num+1
                    timeind=1.
            if (R <= 0.):
                break
        if (R <= 0.):
            break            
            timeind=timeind+1
    return num

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

for j in range(1,99999,1):

    timeabs.append(j*deltat2)
    


    mcore=(mcore+Mdotdisk*(5.97e27/(365*3600*24))*deltat2)

    init = (d0*boltz*T0)/(mudisk) , T0
    Rcore=(3*mcore/(4*3.14*rhoc))**(1./3)
    Bondi = (ggrav*mcore/(boltz*T0/mudisk))/Rcore
    mdottot=Mdotdisk*(5.97e27/(365*3600*24))
    radgradp=(3*mdottot)/(64*3.14*stephboltz*Rcore)
    Qep=(0.3*2*3.14*psize)/(0.289)
    dMassg=np.zeros(nteps)
    x = np.logspace(12,8.7,nteps)
    radgradtotind=[]
    adiagradind=[]


    sol=odeint(g, init, x)  

    Density = (((sol[:,0]*mu)/(boltz*sol[:,1])))
    Temperature = (sol[:,1])/T0
    Pressure=sol[:,0]

#    for i in range(1,nteps,1):
#        dMassg[i]=((4/3)*3.14*Density[i]*((x[i-1]-x[i])**3))
    dMassg=((4/3)*3.14*Density*((np.diff(x,n=0))**3))        
        
        
    if (Rini > 0):
        xfin.append(x[subR(Rini)]/Rcore)
#        dMassice[subR(Rini)]=dMassice[subR(Rini)] + (4/3)*3.14*rhos*(Rini)**3
        dMassice[subR(Rini)]=dMassice[subR(Rini)] + 1e-5*(5.97e27)*deltat2
#        print subR(Rini),Rini
    mu=(1./(((dMassg/(dMassg+dMassice))/2.2)+(((dMassice)/(dMassg+dMassice))/18.))) * 1.67e-24
#    print mu
    mu[np.isnan(mu)] = 2.34 * 1.67e-24

    for i in range(1,nteps-1,1):
        radgradtot2[i]=((radgradp*(sol[i,0]/((sol[i,1])**4.))*((1e-8*(((sol[i,0]*mu[i])/(sol[i,1]*boltz))**(2./3))*(sol[i,1]**(3.))) +
                         ((min(2.,Qep*sol[i,1]))*3*Zgr/(4*rhos*psize)))))
    #print mu2[np.where(mu2>2.2)]
    #mugrad=((np.log(mu[i])-np.log(mu[i-1]))/(np.log(Pressure[i])-np.log(Pressure[i-1])))
    mugrad=(np.gradient(np.log(mu)))/(np.gradient(np.log(Pressure)))
    mugrad=(-1.)*np.abs(mugrad)
    ledouxgrad=(radgradtot2-mugrad)
#    print len(radgradtot2), len(mugrad)

    
    #print (adiagrad-ledouxgrad)
#    print len(radgradtotind), len(adiagradind),len(radgradtot2)
   
    
    if (mcore >= 5*(5.97e27)):
        break
#print mu